<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [103]:
from scipy.special import comb

In [104]:
def get_fail_prob( lcomb, nrs, perr ):
    a = []
    for nopen in lcomb:
        p=0.0
        for nsel in np.arange( nopen, nrs, -1 ):
            p+= comb( nopen, nsel) * perr**nsel * (1-perr)**(nopen-nsel)
        a += [p]
    return a


In [105]:
def get_not_pass_prob( lprob ):
    if len(lprob) == 1:
        return lprob[0]
    else:
        recp = get_not_pass_prob( lprob[1:])
        return lprob[0] + recp - lprob[0]*recp


In [106]:
lprob = get_fail_prob( [3,2,1,2,5,6,1], 2, 1e-6)
get_not_pass_prob( lprob )

3.0999940000041991e-17

$$
\phi(N,M,(x_1,x_2,\dots,x_n)) = N\ {}_MC_{x_1}\phi(N-1, M-x_1,(x_2,x_3,\dots,x_n)) 
$$

In [1]:
import itertools
NUM_SYMBOL = 64
NUM_CHIP = 8
NUM_PLANE = 2
NUM_SIB = NUM_SYMBOL // ( NUM_CHIP * 2 ) # number of SIB in LIB
NUM_BLK = 10
numcor = 2

bb_tbl = np.random.randn( 100,2 ) * 4
bb_tbl = bb_tbl.clip( 0, 5).astype(int)
bb_tbl = bb_tbl[ np.sum( bb_tbl, axis=1) > 0 ]


In [2]:
def get_bb_tbl_libb2( bb_mask_libb2, bb_tbl_sibb ):
    ''' Make bb_tbl_libb2 from bb_tbl_sibb considering BB selection based on bb_mask_libb2
        bb_mask_lib2 : True:select BB, False:select normal Blk. array(NUM_SIB,len_bb_sibb)
        bb_tbl_sib   : number of BB each Blk in SIBB has
    '''
    bb_tbl_libb2 = np.zeros_like( bb_mask_libb2, dtype=np.int32 )
    bb_tbl_sibb_tmp = bb_tbl_sibb.copy()
    for sib in range(bb_mask_libb2.shape[0]):
#        print( bb_tbl_sibb_tmp)
        bb_tbl_libb2[sib] = bb_tbl_sibb_tmp
        bb_tbl_sibb_tmp -= bb_mask_libb2[sib].astype(int)

    return bb_tbl_libb2


In [3]:
def get_pass_prob_libb2( bb_mask_libb2, bb_tbl_libb2 ):
    ''' Get Pass probability
    '''
    pass_prob_libb2 = np.empty_like( bb_mask_libb2, dtype=np.float )
    left_blk_num_libb2 = np.empty_like( bb_mask_libb2, dtype=np.int32)
    left_blk_num_libb2[0] = NUM_BLK
    for i in range( 1, left_blk_num_libb2.shape[0] ):
        left_blk_num_libb2[i] = left_blk_num_libb2[i-1] -1
#    print( left_blk_num_libb2 )
    pass_prob_libb2[bb_mask_libb2] = bb_tbl_libb2[bb_mask_libb2] \
                                     / left_blk_num_libb2[bb_mask_libb2] # Select BB
    pass_prob_libb2[~bb_mask_libb2] = 1.0 - bb_tbl_libb2[~bb_mask_libb2] \
                                            / left_blk_num_libb2[~bb_mask_libb2] # Select nomarl Blk

    return pass_prob_libb2

In [12]:
#def get_prob( bb_chips, bb_tbl, numcorbb ):
#    ''' Get PASS probability
#        bb_chips : list of chips including BB in SIB
#        bb_tbl   : number of BB per plane for each BB chip
#        numcorbb   : number of RS correctable error symbols
#    '''
probs = {}
prob_sum = 0.0
bb_tbl_sibb = bb_tbl[ bb_chips].ravel()
bb_len_sibb = len(bb_tbl_sibb)
bb_len_libb = bb_len_sibb * NUM_SIB
for numbb in range(numcorbb+1):
    probs[numbb] = {}    
    for bbpos in itertools.combinations( range(bb_len_libb), numbb ):
        bb_mask_libb = np.zeros((bb_len_libb),dtype=np.bool)
        bb_mask_libb[list(bbpos)] = True
        bb_mask_libb2 = bb_mask_libb.reshape( NUM_SIB, bb_len_sibb )
        if np.any( bb_mask_libb2.sum(axis=0) > bb_tbl_sibb ):
            continue
        bb_tbl_libb2 = get_bb_tbl_libb2( bb_mask_libb2, bb_tbl_sibb )
#        print(bb_tbl_libb2)
        pass_prob_libb2 = get_pass_prob_libb2( bb_mask_libb2, bb_tbl_libb2 )

        pass_prob_prod = pass_prob_libb2.prod()
        probs[numbb][bbpos] = pass_prob_prod
        prob_sum += pass_prob_prod
##    return prob_sum, probs
#    return 1-prob_sum, probs


In [11]:
bb_tbl[2] = [2,0]
bb_tbl[4] = [1,0]
#bb_chips = [ 2, 4 ]
#bb_chips = [ 2 ]
numcorbb = 2

In [13]:
print(prob_sum)

0.946666666667


In [14]:
print(probs)

{0: {(): 0.20000000000000001}, 1: {(0,): 0.080000000000000016, (2,): 0.03333333333333334, (4,): 0.080000000000000016, (6,): 0.033333333333333333, (8,): 0.080000000000000016, (10,): 0.033333333333333333, (12,): 0.080000000000000002, (14,): 0.033333333333333333}, 2: {(0, 2): 0.013333333333333338, (0, 4): 0.013333333333333334, (0, 6): 0.013333333333333334, (0, 8): 0.013333333333333334, (0, 10): 0.013333333333333334, (0, 12): 0.013333333333333332, (0, 14): 0.013333333333333332, (2, 4): 0.013333333333333338, (2, 8): 0.013333333333333338, (2, 12): 0.013333333333333336, (4, 6): 0.013333333333333334, (4, 8): 0.013333333333333334, (4, 10): 0.013333333333333334, (4, 12): 0.013333333333333332, (4, 14): 0.013333333333333332, (6, 8): 0.013333333333333336, (6, 12): 0.013333333333333332, (8, 10): 0.013333333333333336, (8, 12): 0.013333333333333334, (8, 14): 0.013333333333333336, (10, 12): 0.013333333333333332, (12, 14): 0.013333333333333331}}


In [116]:
prob_sum, probs = get_prob( bb_chips, bb_tbl, numcorbb)

[1 1]
[1 1]
[1 1]
[1 1]


In [6]:
%qtconsole

In [22]:
0.8*0.9*0.77777778*0.88888889*0.75*0.875*0.28571429*0.14285714


0.013333333321428569